The source dataset consists of a set of audio and xml files describing them. It is necessary to generate data to verify proper operation of the future system for the detection of critical situations.

In [1]:
import numpy as np
import pandas as pd
from xml.dom import minidom

In [2]:
x = minidom.parse("./XML/testing/00001.xml")
events = x.getElementsByTagName('events')[0]
itemlist = events.getElementsByTagName('item')

In [3]:
time_ranges = []
for item in itemlist:
    startsecond = float(item.getElementsByTagName("STARTSECOND")[0].firstChild.nodeValue)
    endsecond = float(item.getElementsByTagName("ENDSECOND")[0].firstChild.nodeValue)
    classid = int(item.getElementsByTagName("CLASS_ID")[0].firstChild.nodeValue)
    time_ranges.append((startsecond, endsecond, classid))

In [60]:
file_length = int(time_ranges[len(time_ranges) - 1][1]) + 1  #seconds
freq_rate = 32000
frameSize = 1024
hopSize = int(frameSize / 2)

frames_raw = []
for i, start in enumerate(range(0, file_length * freq_rate, hopSize)):
    end = start + frameSize
    start_ms = start / freq_rate
    end_ms = end / freq_rate
    event = None
    for time_range in time_ranges:
        if time_range[0] < start_ms < time_range[1]:
            event = time_range[2]
            break
        elif time_range[0] < end_ms < time_range[1]:
            event = time_range[2]
            break
    frames_raw.append((i, event))

In [75]:
#1 - neutral
#2 - glass
#3 - gunshot
#4 - scream

big_window_size = 32

frames_big_df = pd.DataFrame(columns=['frame_num', 'neutral', 'glass', 'gunshot', 'scream'])
for frame_num, start_frame in enumerate(range(0, len(frames_raw), int(big_window_size / 2))):
    frame_big = frames_raw[start_frame:start_frame + big_window_size]
    count = np.zeros(4, dtype=np.int)
    for f in frame_big:
        if f[1] is None:
            count[0] += 1
        elif f[1] == 2:
            count[1] += 1
        elif f[1] == 3:
            count[2] += 1
        elif f[1] == 4:
            count[3] += 1
    max_classid = np.argmax(count)
    count_val = count[max_classid]
    max_classid += 1 #neutral is '1', not '0'
    if (count_val > (int(big_window_size / 2))):
        frames_big_df.loc[len(frames_big_df)] = [frame_num, (1 == max_classid), (2 == max_classid),
                                                (3 == max_classid), (4 == max_classid)]

In [79]:
frames_big_df [20:30]

,frame_num,neutral,glass,gunshot,scream
20,20,False,True,False,False
21,21,False,True,False,False
22,22,True,False,False,False
23,23,True,False,False,False
24,24,True,False,False,False
25,25,True,False,False,False
26,26,True,False,False,False
27,27,True,False,False,False
28,28,True,False,False,False
29,29,True,False,False,False


In [81]:
frames_big_df.to_csv('./csv/validation.csv', sep='\t')